In [1]:
import networkx as nx
import matplotlib as mplot
import urllib
import pandas as pd
import numpy as np
%matplotlib inline


In [2]:
# Import the raw data file
url = 'https://github.com/bkreis84/Web-Analytics/blob/master/bankmergerdata-xpt.xpt?raw=true'

In [3]:
#use pandas to read a sas file
g = pd.read_sas(url)

In [4]:
#convert to dataframe
df = pd.DataFrame(g)

In [5]:
# Look at merger date on or after 2000
df = df[df['MERGE_DT'] >= 20000101]

#Pull out only the instances where the bank being acquired ceased to exist -didn't just sell some or most of their assets
codes = [50,1,9]
df = df[df['MERGE_CD'].isin(codes)]

In [6]:
df.count()

MERGE_DT    6237
MERGE_CD    6237
CODE        6237
NON_ID      6237
NON_SURV    6237
NON_CITY    6237
NON_ST      6237
NON_TYP     6237
NON_TID     6237
NON_TOP     6237
SURV_ID     6237
SURV_NM     6237
SURV_CTY    6237
SURV_ST     6237
SURV_TYP    6237
SURV_TID    6237
SURV_TOP    6237
ACT_MTHD    6237
dtype: int64

In [7]:
df.head()

,MERGE_DT,MERGE_CD,CODE,NON_ID,NON_SURV,NON_CITY,NON_ST,NON_TYP,NON_TID,NON_TOP,SURV_ID,SURV_NM,SURV_CTY,SURV_ST,SURV_TYP,SURV_TID,SURV_TOP,ACT_MTHD
8,20070817,1,5.397605e-79,439,PEOPLES BK,BLACKSHEAR,GA,NMB,1.086748e+06,LIBERTY SHARES,507330,HERITAGE BK,HINESVILLE,GA,NMB,1086748,LIBERTY SHARES,1.000000e+00
12,20000325,1,5.397605e-79,608,VERMONT NB,BRATTLEBORO,VT,NAT,1.114605e+06,CHITTENDEN CORP,40305,CHITTENDEN TC,BURLINGTON,VT,NMB,1114605,CHITTENDEN CORP,1.000000e+00
19,20070423,1,2.000000e+00,1146,PULASKI B&TC,LITTLE ROCK,AR,SMB,2.291914e+06,IBERIABANK CORP,369378,PULASKI B&TC,LITTLE ROCK,AR,SAL,2291914,IBERIABANK CORP,5.397605e-79
22,20100601,1,5.397605e-79,1632,SEA ISLAND BK,STATESBORO,GA,NMB,1.078846e+06,SYNOVUS FC,395238,SYNOVUS BK,COLUMBUS,GA,NMB,1078846,SYNOVUS FC,1.000000e+00
30,20020322,1,5.397605e-79,1959,SECURITY ST BK,BIRD CITY,KS,NMB,5.397605e-79,N/A,805250,FIRST NB,GOODLAND,KS,NAT,2267731,FIRST NAT BSHRS ESOP & 401 K T,2.000000e+00


In [8]:
# Sort by which companies had the most acquistions and show the top 10
df['1'] = 1
df = df.replace('N/A', np.nan)
group = df[['SURV_TOP','1']]
group = group.groupby('SURV_TOP').sum()
group = group.sort_values(by='1', ascending=False)
group[:10]

,1
SURV_TOP,
WELLS FARGO & CO,116
BB&T CORP,103
U S BC,56
SYNOVUS FC,50
MARSHALL & ILSLEY CORP,41
SUNTRUST BK,39
FNB CORP,39
HARRIS FC,35
BANK OF AMER CORP,33


In [9]:
#subset to the data we need and get rid of rows that contain NaN values
sub = df[['SURV_ID', 'SURV_NM', 'SURV_TYP', 'SURV_ST', 'NON_ID', 'NON_SURV', 'NON_TYP', 'NON_ST', 'CODE', 'MERGE_CD' ]]
sub.count()
sub = sub.dropna()
sub.count()

SURV_ID     6237
SURV_NM     6237
SURV_TYP    6237
SURV_ST     6237
NON_ID      6237
NON_SURV    6237
NON_TYP     6237
NON_ST      6237
CODE        6237
MERGE_CD    6237
dtype: int64

In [10]:
#create the nodes, starting with the non surviving institutions
non = sub[['NON_ID', 'NON_SURV', 'NON_TYP', 'NON_ST']]
non = non.drop_duplicates('NON_SURV')
non.columns = ['ID', 'Name', 'Type', 'State']
non.count()


ID       5096
Name     5096
Type     5096
State    5096
dtype: int64

In [11]:
#Find the acquiring banks for our nodes
surv = sub[['SURV_ID', 'SURV_NM', 'SURV_TYP', 'SURV_ST']]
surv = surv.drop_duplicates('SURV_ID')
surv.columns = ['ID', 'Name', 'Type', 'State']
surv.count()

ID       2340
Name     2340
Type     2340
State    2340
dtype: int64

In [12]:
#combined nodes
nodes = pd.concat([surv, non], ignore_index=True)
nodes.count()

ID       7436
Name     7436
Type     7436
State    7436
dtype: int64

In [13]:
#There may be banks that made acquisitions that no longer exist. We need to remove those duplicates.
nodes = nodes.drop_duplicates('ID')
nodes.count()

ID       6823
Name     6823
Type     6823
State    6823
dtype: int64

In [14]:
#create the empty graph and add nodes and atributes 
G = nx.DiGraph()

for i in range(len(nodes)):
    G.add_node(nodes.iloc[i]['Name'], ID=nodes.iloc[i]['ID'], Type=nodes.iloc[i]['Type'], 
               State=nodes.iloc[i]['State'])
    


In [15]:
# create edges using the earlier dataframe
for i in range(len(sub)):
    G.add_edge(sub.iloc[i]['SURV_NM'], sub.iloc[i]['NON_SURV'], MergerID=sub.iloc[i]['CODE'], 
               Code=sub.iloc[i]['MERGE_CD'])
    

In [16]:
#Quick test 
G.node['SYNOVUS BK']


{'ID': 395238.0, 'State': 'GA', 'Type': 'NMB'}

In [17]:
G.edge['SYNOVUS BK']['SEA ISLAND BK']


{'Code': 1.0, 'MergerID': 5.3976053469340279e-79}

In [ ]:
Gephi image of network below

![Image of Yaktocat](https://github.com/bkreis84/Web-Analytics/blob/master/Network.jpg?raw=true)

In [18]:
 nx.write_pajek(G,"Banks.net") # save the file and it can be read by most any SNA package

In [19]:
#Test betweenness and assign it as a node attribute
bw = nx.betweenness_centrality(G)
nx.set_node_attributes(G, 'betweenness', bw)

In [20]:
#sort and display the top 10
bwsort = bw.items()
bwsort.sort(key=lambda x:x[1], reverse=True)
bwsort[:10]

[('FIRST NB', 0.0028166456121925675),
 ('FIRST BK', 0.0016618575792614438),
 ('CITIZENS ST BK', 0.001489977218413474),
 ('FIRST ST BK', 0.001094100588213008),
 ('CITIZENS BK', 0.0008901236631546599),
 ('FIRST CMNTY BK', 0.0008543245752224134),
 ('COMMUNITY BK', 0.0005580057375690764),
 ('STATE B&TC', 0.0004418189319460556),
 ('FIRST B&TC', 0.0004332266269938783),
 ('PEOPLES BK', 0.0004318066802422783)]

In [21]:
#test the eigenvector centrality and assign it as a node attribute
ev = nx.eigenvector_centrality(G)
nx.set_node_attributes(G, 'eigenvector_centrality', ev)

In [22]:
evsort = ev.items()
evsort.sort(key=lambda x:x[1], reverse=True)
evsort[:10]

[('FIRST NB', 0.29593738910472517),
 ('FIRST ST BK', 0.24611569105371),
 ('CITIZENS BK', 0.18724668088855678),
 ('SECURITY ST BK', 0.180825305029966),
 ('NATIONAL BK', 0.14004174851672702),
 ('CITIZENS NB', 0.12545680004460963),
 ('FARMERS BK', 0.11631730347074468),
 ('CITY ST BK', 0.11631714833774677),
 ('CITIZENS B&TC', 0.1003404130939913),
 ('HOME ST BK', 0.09520810380056823)]

In [23]:
G.node['FIRST NB']

{'ID': 22954.0,
 'State': 'MN',
 'Type': 'NAT',
 'betweenness': 0.0028166456121925675,
 'eigenvector_centrality': 0.29593738910472517}

In [24]:
#Test the degree and assign it as a node attribute
dc = nx.degree_centrality(G)
nx.set_node_attributes(G, 'degree_centrality', dc)

In [25]:
dcsort = dc.items()
dcsort.sort(key=lambda x:x[1], reverse=True)
dcsort[:10]

[('FDIC', 0.02947076068507946),
 ('FIRST NB', 0.018207066810677363),
 ('FIRST ST BK', 0.013115259990742168),
 ('BRANCH BKG&TC', 0.012189476932572134),
 ('WELLS FARGO BK NA', 0.010492207992593735),
 ('U S BK NA', 0.007097670112636938),
 ('FIRST BK', 0.006480481407190248),
 ('SUNTRUST BK', 0.00601758987810523),
 ('CITIZENS BK', 0.00601758987810523),
 ('M&I MARSHALL & ILSLEY BK', 0.00540040117265854)]

In [26]:
#Get the node attributes we need to do some statistical testing
org = nx.get_node_attributes(G,'Type')
eigen = nx.get_node_attributes(G,'eigenvector_centrality')
dcen = nx.get_node_attributes(G,'degree_centrality')



In [27]:
#merge the data
btype = pd.DataFrame(org.items(), columns=['Bank', 'Type'])
egv = pd.DataFrame(eigen.items(), columns=['Bank', 'Eigenvector'])
dcf = pd.DataFrame(dcen.items(), columns=['Bank', 'Degree Centrality'])

In [28]:
x = btype.merge(egv)
x = x.merge(dcf)

In [29]:
x.head()

,Bank,Type,Eigenvector,Degree Centrality
0,FARMERS ST BK OF JETMORE KS,NMB,0.000000e+00,0.000154
1,STATE BK OF ROGERS,NMB,0.000000e+00,0.000154
2,FIRST BK OF HASKELL,SMB,0.000000e+00,0.000154
3,BANK OF LARAMIE NA,NAT,7.091794e-08,0.000154
4,SIUSLAW BK,NMB,0.000000e+00,0.000154


In [30]:
import scipy.stats as stats

In [31]:
group_by_type = x.groupby('Type')
means = group_by_type.mean()
means

,Eigenvector,Degree Centrality
Type,,
AGI,0.000000e+00,0.000154
BHC,4.095218e-03,0.000154
CPB,9.402000e-10,0.000178
DEO,1.442548e-03,0.000246
DPS,3.893600e-05,0.000154
EDB,0.000000e+00,0.000154
FCU,0.000000e+00,0.000154
FHD,7.387286e-10,0.000154
FNC,1.187242e-09,0.000154


##### Null Hypothesis: There is not a statistically significant difference between the type of bank and the degree centrality of the populations mean as it pertains to mergers and acquisitions

##### Alternative Hypothesis:  There is a statistically significant difference between the type of bank and the degree centrality of the populations mean as it pertains to mergers and acquisitions

In [32]:
# Find the top 10 for degree centrality so we can test the following
dc10 = means.sort_values(['Degree Centrality'], ascending=False)

dc10[:10]

,Eigenvector,Degree Centrality
Type,,
SMB,2.331903e-03,0.000338
NAT,2.404230e-03,0.000335
NMB,2.277348e-03,0.000291
FSB,6.693665e-04,0.000247
DEO,1.442548e-03,0.000246
SSB,1.402226e-03,0.000233
MTC,2.350043e-03,0.000199
SAL,1.003945e-03,0.000186
CPB,9.402000e-10,0.000178


In [33]:
bhcDC = x[x.Type.isin(['BHC'])]
natDC = x[x.Type.isin(['NAT'])]
mtcDC = x[x.Type.isin(['MTC'])]
smbDC = x[x.Type.isin(['SMB'])]
nmbDC = x[x.Type.isin(['NMB'])]
deoDC = x[x.Type.isin(['DEO'])]
ssbDC = x[x.Type.isin(['SSB'])]
salDC = x[x.Type.isin(['SAL'])]
fsbDC = x[x.Type.isin(['FSB'])]
sbdDC = x[x.Type.isin(['SBD'])]

In [34]:
bhc1 = bhcDC['Degree Centrality']
nat1 = natDC['Degree Centrality']
mtc1 = mtcDC['Degree Centrality']
smb1 = smbDC['Degree Centrality']
nmb1 = nmbDC['Degree Centrality']
deo1 = deoDC['Degree Centrality']
ssb1 = ssbDC['Degree Centrality']
sal1 = salDC['Degree Centrality']
fsb1 = fsbDC['Degree Centrality']
sbd1 = sbdDC['Degree Centrality']

In [35]:
stats.f_oneway(bhc1, nat1, mtc1, smb1, nmb1, deo1, ssb1, sal1, fsb1, sbd1)

F_onewayResult(statistic=2.6161431040679242, pvalue=0.0051420160090256085)

#Our P-value is .0051 indicating that there is less than a 1% chance that the differences in degree centrality could have occured by chance. We therefore rject the null hypothesis and acept the alternative hypothesis, indicating that the type of bank may have an influence on the degree centrality.